<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Amazon_Domain_Pre_training_5M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This takes a 5Mreview dataset, compiles the model,  saves checkpoints every and pushes the final model to the hub.

# Results


# Details
* Uses batch size of 32 and gradient accumulation of 11
  * Gradient Updates every 352 rows (32 * 11)
  * Slightly more than 2112 review batch size (more than 352 * 6)
* Uses 1 epoch
* Dataset - 5M rows train
* Saves checkpoint every 500 steps
  * Only has one current save on the google drive
  * Saves checkpoint and model at end in
* Compile creates warning but it seems ok: popen_fork.py:66: RuntimeWarning: os.fork()




# Setup

In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/AdaptOrDie'

Mounted at /content/drive
/content/drive/MyDrive/AdaptOrDie


In [2]:
!pip install datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━

In [3]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00


In [4]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [5]:
from datasets import load_dataset
dataset = load_dataset("BigTMiami/amazon_split_25M_reviews_20_percent_condensed")
print(dataset)
print_gpu_utilization()

Generating train split:   0%|          | 0/862683 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8360 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 862683
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8360
    })
})
GPU 448 MB


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
block_size = tokenizer.model_max_length
print(f"block_size:{block_size}")
print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

block_size:512
GPU 448 MB


In [7]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_config(config)
print_gpu_utilization()

GPU 448 MB


# Train

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="amazon_pretraining_5M_model",
    learning_rate=0.0005, # Paper for DAPT training
    per_device_train_batch_size=32, # 346,m - TRYING SMALLER BATCH SIZE
    per_device_eval_batch_size=32, #346,m - TRYING SMALLER BATCH SIZE
    gradient_accumulation_steps=11, # Creates effective batch of 32*11*6 = 2112
    num_train_epochs=1, # Just one epoch for one pass on data
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="steps",
    save_steps=500, # Checkpoint saves every 500 steps / every 1,000,000 reviews ( 500 * 10 * 34 * 6)
    save_total_limit=1, # Saves latest checkpoint
    torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_steps=100, # Logs training progress
)

In [9]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


GPU 1399 MB


In [10]:
eval_results = trainer.evaluate()
non_trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Non Trained Eval Loss: {non_trained_eval_loss:.2f}")
print(eval_results)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


GPU 7967 MB
Non Trained Eval Loss: 11.05
{'eval_loss': 11.053717613220215, 'eval_runtime': 82.4073, 'eval_samples_per_second': 101.447, 'eval_steps_per_second': 3.179}


In [11]:
results = trainer.train()
print_gpu_utilization()
print(results)

Epoch,Training Loss,Validation Loss
0,7.066900,7.072030


GPU 40475 MB
TrainOutput(global_step=2450, training_loss=7.1279049744897955, metrics={'train_runtime': 8789.1315, 'train_samples_per_second': 98.153, 'train_steps_per_second': 0.279, 'total_flos': 2.270401355464704e+17, 'train_loss': 7.1279049744897955, 'epoch': 1.0})


In [12]:
eval_results = trainer.evaluate()
trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Trained Eval Loss: {trained_eval_loss:.2f}")
print(eval_results)

GPU 40475 MB
Trained Eval Loss: 7.07
{'eval_loss': 7.068391799926758, 'eval_runtime': 37.2685, 'eval_samples_per_second': 224.318, 'eval_steps_per_second': 7.03, 'epoch': 1.0}


In [13]:
trainer.save_model()

events.out.tfevents.1712626786.5ec7b088f7c8.293.0:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1712635613.5ec7b088f7c8.293.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [14]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
